# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sqlalchemy import create_engine

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
# load data from database
def load_data():
    '''
    Argument
    Returns
    '''
    engine = create_engine('sqlite:///DisasterResponse.db')

    df = pd.read_sql_table('Messages', engine)

    X = df['message'].copy()
    
    y = df.columns
    y = y.drop(['id', 'message', 'original', 'genre'])
    y = df[y].copy()

    return X, y

In [13]:
X, y = load_data()

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    '''
    Argument
    Returns
    '''

    # Normalise text
    text = text.lower()

    # Remove URLs
    url_mask = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    urls = re.findall(url_mask, text)
    for url in urls:
        text = text.replace(url, "urlplaceholder")

    # Remove punctuation
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

    # Tokenise text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Lemmatisation
    lemmatiser = WordNetLemmatizer()
    lemmed_tokens = [lemmatiser.lemmatize(token) for token in stemmed_tokens]
    lemmed_tokens = [lemmatiser.lemmatize(token, pos='v') for token in lemmed_tokens]

    clean_tokens = lemmed_tokens

    return clean_tokens

In [17]:
tokenize('Hello, dear friend! My name is Sajid. We went to https://sajidsarker.github.io')

['hello', 'dear', 'friend', 'name', 'sajid', 'go', 'urlplacehold']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

def model_pipeline():
    pipeline = Pipeline([
        #('feature', FeatureUnion([

        ('nlp_pipeline', Pipeline([
            ('vectorise', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        #    ('', fn())
            
        #])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline, pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model, pipeline_params = model_pipeline()

model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vectorise', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def display_results(y_test, y_pred):
    '''
    Argument
    Returns
    '''
    for column in y_test.columns:
        print(classification_report(y_true[column], y_pred[column]))
    #class_labels = np.unique(y_pred)
    #confusion_matrix(y_test, y_pred, class_labels)
    #f1 = 0
    #precision = 0
    #recall = 0
    
    #classification_report

    #print('Class Labels:', class_labels)
    #print('Confusion Matrix:\n', confusion_matrix)
    #print('F1 Score:', f1)
    #print('Precision:', precision)
    #print('Recall:', recall)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
def cross_validate_model(pipeline):
    parameters = {
        '': ,
        '': (
            {'': },
            {'': },
            {'': }
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cross_validation = build_model(pipeline=model)

y_pred = model.predict(X_test)

display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def model_pipeline2():
    pass

### 9. Export your model as a pickle file

In [1]:
import pickle

def pickle_model(filename, model):
    output_file = open(filename, 'wb')
    pickle.dump(model, output_file)
    output_file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.